## Preprocess Data for VAE

The aim of this notebook is to translate NetCDF files (.nc) of three daily climate variables (maximum temperature, precipitations, wind) to a numpy 3D-array. This output array can easily be read for training and evaluating the Convolutional Variational AutoEncoder model.

#### 0. Libraries

In [1]:
import numpy as np
import xarray as xr
import cftime
import csv
import pandas as pd
from datetime import datetime

#### 1. Load Data to xarrays

In [7]:
# Historical Datasets
# regrouped by climate variable

temp_50 = xr.open_dataset("../data/tasmax_day_CMCC-ESM2_historical_r1i1p1f1_gn_19500101-19741231.nc")
temp_75 = xr.open_dataset("../data/tasmax_day_CMCC-ESM2_historical_r1i1p1f1_gn_19750101-19991231.nc")
temp_00 = xr.open_dataset("../data/tasmax_day_CMCC-ESM2_historical_r1i1p1f1_gn_20000101-20141231.nc")
temp_histo = xr.concat([temp_50, temp_75], "time")
temp_histo = xr.concat([temp_histo, temp_00], "time")

prcp_50 = xr.open_dataset("../data/pr_day_CMCC-ESM2_historical_r1i1p1f1_gn_19500101-19741231.nc")
prcp_75 = xr.open_dataset("../data/pr_day_CMCC-ESM2_historical_r1i1p1f1_gn_19750101-19991231.nc")
prcp_00 = xr.open_dataset("../data/pr_day_CMCC-ESM2_historical_r1i1p1f1_gn_20000101-20141231.nc")
prcp_histo = xr.concat([prcp_50, prcp_75], "time")
prcp_histo = xr.concat([prcp_histo, prcp_00], "time")

wind_50 = xr.open_dataset("../data/sfcWind_day_CMCC-ESM2_historical_r1i1p1f1_gn_19500101-19741231.nc")
wind_75 = xr.open_dataset("../data/sfcWind_day_CMCC-ESM2_historical_r1i1p1f1_gn_19750101-19991231.nc")
wind_00 = xr.open_dataset("../data/sfcWind_day_CMCC-ESM2_historical_r1i1p1f1_gn_20000101-20141231.nc")
wind_histo = xr.concat([wind_50, wind_75], "time")
wind_histo = xr.concat([wind_histo, wind_00], "time")

In [8]:
# Projection Datasets
# regrouped by climate variable

temp_40 = xr.open_dataset("../data/tasmax_day_CMCC-ESM2_ssp585_r1i1p1f1_gn_20400101-20641231.nc")
temp_65 = xr.open_dataset("../data/tasmax_day_CMCC-ESM2_ssp585_r1i1p1f1_gn_20650101-20891231.nc")
temp_90 = xr.open_dataset("../data/tasmax_day_CMCC-ESM2_ssp585_r1i1p1f1_gn_20900101-21001231.nc")
temp_proj = xr.concat([temp_40, temp_65], "time")
temp_proj = xr.concat([temp_proj, temp_90], "time")

prcp_40 = xr.open_dataset("../data/pr_day_CMCC-ESM2_ssp585_r1i1p1f1_gn_20400101-20641231.nc")
prcp_65 = xr.open_dataset("../data/pr_day_CMCC-ESM2_ssp585_r1i1p1f1_gn_20650101-20891231.nc")
prcp_90 = xr.open_dataset("../data/pr_day_CMCC-ESM2_ssp585_r1i1p1f1_gn_20900101-21001231.nc")
prcp_proj = xr.concat([prcp_40, prcp_65], "time")
prcp_proj = xr.concat([prcp_proj, prcp_90], "time")

wind_40 = xr.open_dataset("../data/sfcWind_day_CMCC-ESM2_ssp585_r1i1p1f1_gn_20400101-20641231.nc")
wind_65 = xr.open_dataset("../data/sfcWind_day_CMCC-ESM2_ssp585_r1i1p1f1_gn_20650101-20891231.nc")
wind_90 = xr.open_dataset("../data/sfcWind_day_CMCC-ESM2_ssp585_r1i1p1f1_gn_20900101-21001231.nc")
wind_proj = xr.concat([wind_40, wind_65], "time")
wind_proj = xr.concat([wind_proj, wind_90], "time")

#### 2. Restrict to a Geospatial Square

In [21]:
sq32_west_europe = {
    "min_lon": -10,
    "max_lon": 29,
    "min_lat": 36,
    "max_lat": 66
}

In [22]:
def xr_to_ndarray(xr_dset: xr.Dataset, 
                  sq_coords: dict
                 ) -> (np.ndarray, np.array, str):
    """
    Convert xarray dataset it to a cropped square ndarray.
    :param sq_coords: spatial coordinates of the crop
    """
    xr_dset.coords['lon'] = (xr_dset.coords['lon'] + 180) % 360 - 180
    xr_dset = xr_dset.sortby(xr_dset.lon)
    xr_dset = xr_dset.sel(
        lon = slice(sq_coords['min_lon'], sq_coords['max_lon']),
        lat = slice(sq_coords['min_lat'], sq_coords['max_lat'])
    )
    time_list = np.array(xr_dset['time'])
    n_t = len(time_list)
    n_lat = len(xr_dset.coords['lat'])
    n_lon = len(xr_dset.coords['lon'])
    nd_dset = np.ndarray((n_t, n_lat, n_lon, 1), dtype="float32")
    climate_variable = xr_dset.attrs['variable_id']
    nd_dset[:, :, :, 0] = xr_dset[climate_variable][:, :, :]
    nd_dset = np.flip(nd_dset, axis=1)
    
    return nd_dset, time_list

#### 3. Normalize

In [23]:
def get_extrema(histo_dataset: np.ndarray,
                proj_dataset: np.ndarray) -> np.array:
    # compute global extrema over past and future
    global_min = min(np.min(histo_dataset), np.min(proj_dataset))
    global_max = max(np.max(histo_dataset), np.max(proj_dataset))
    return np.array([global_min, global_max])

In [24]:
def normalize(nd_dset: np.ndarray, 
              extrema: np.array
             ) -> np.ndarray:
    norm_dset = (nd_dset-extrema[0])/(extrema[1]-extrema[0])
    return norm_dset

#### 4. Split Historical Data into Train and Test Datasets

Train the network on most of the historical data, but keep some to test the model performance on new data points.

In [25]:
def split_train_test(nd_dset: np.ndarray,
                     time_list: np.array,
                     train_proportion: float = 0.8
                    ) -> (np.ndarray, np.ndarray, np.array, np.array):
    len_train = int(len(nd_dset)*train_proportion)
    train_data = nd_dset[:len_train]
    test_data = nd_dset[len_train:]
    train_time = time_list[:len_train]
    test_time = time_list[len_train:]
    return train_data, test_data, train_time, test_time

#### 5. Combine into a 3D-Array

In [26]:
def ndarray_to_3d(temp_dset: np.ndarray,
                 prcp_dset: np.ndarray,
                 wind_dset: np.ndarray
                 ) -> np.ndarray:
    
    n_t = np.shape(temp_dset)[0]
    n_lat = np.shape(temp_dset)[1]
    n_lon = np.shape(temp_dset)[2]
    
    # combine all variables on a same period to a new 3D-array
    total_dset = np.zeros((n_t, n_lat, n_lon, 3), dtype="float32")
    total_dset[:,:,:,0] = temp_dset.reshape(n_t,n_lat,n_lon)
    total_dset[:,:,:,1] = prcp_dset.reshape(n_t,n_lat,n_lon)
    total_dset[:,:,:,2] = wind_dset.reshape(n_t,n_lat,n_lon)
    
    return total_dset

#### 6. Full Preprocessing

In [31]:
def preprocess_3d(temp_histo: xr.Dataset,
                 temp_proj: xr.Dataset,
                 prcp_histo: xr.Dataset,
                 prcp_proj: xr.Dataset,
                 wind_histo: xr.Dataset,
                 wind_proj: xr.Dataset,
                 saving_on: bool = True):
    
    # convert historical xarrays to ndarrays for each climate variable
    temp_histo_nd, time_list = xr_to_ndarray(temp_histo, sq32_west_europe)
    prcp_histo_nd, _ = xr_to_ndarray(prcp_histo, sq32_west_europe)
    wind_histo_nd, _ = xr_to_ndarray(wind_histo, sq32_west_europe)

    # projection xarrays to ndarrays
    temp_proj_nd, time_proj = xr_to_ndarray(temp_proj, sq32_west_europe)
    prcp_proj_nd, _ = xr_to_ndarray(prcp_proj, sq32_west_europe)
    wind_proj_nd, _ = xr_to_ndarray(wind_proj, sq32_west_europe)

    # compute extrema for each variable
    temp_extrema = get_extrema(temp_histo_nd, temp_proj_nd)
    prcp_extrema = get_extrema(prcp_histo_nd, prcp_proj_nd)
    wind_extrema = get_extrema(wind_histo_nd, wind_proj_nd)
    extrema = np.array([temp_extrema, prcp_extrema, wind_extrema, temp_extrema, prcp_extrema, wind_extrema])

    # normalize all datasets
    temp_histo_norm = normalize(temp_histo_nd, temp_extrema)
    temp_proj_norm = normalize(temp_proj_nd, temp_extrema)
    prcp_histo_norm = normalize(prcp_histo_nd, prcp_extrema)
    prcp_proj_norm = normalize(prcp_proj_nd, prcp_extrema)
    wind_histo_norm = normalize(wind_histo_nd, wind_extrema)
    wind_proj_norm = normalize(wind_proj_nd, wind_extrema)

    # split historical datasets into train and test ones
    train_temp, test_temp, train_time, test_time = split_train_test(temp_histo_norm,
                                                                    time_list)
    train_prcp, test_prcp, _, _ = split_train_test(prcp_histo_norm, 
                                                   time_list)
    train_wind, test_wind, _, _ = split_train_test(wind_histo_norm,
                                                   time_list)

    # aggregate datasets per time period (3D-ndarrays)
    total_train = ndarray_to_3d(train_temp, train_prcp, train_wind)
    total_test = ndarray_to_3d(test_temp, test_prcp, test_wind)
    total_proj = ndarray_to_3d(temp_proj_norm, 
                               prcp_proj_norm, 
                               wind_proj_norm)

    # save data in input folder
    if saving_on == True:
        np.save("../input/preprocessed_3d_train_data.npy", total_train)
        np.save("../input/preprocessed_3d_test_data.npy", total_test)
        np.save("../input/preprocessed_3d_proj_data.npy", total_proj)
        pd.DataFrame(train_time).to_csv('../input/dates_train_data.csv')
        pd.DataFrame(test_time).to_csv('../input/dates_test_data.csv')
        pd.DataFrame(time_proj).to_csv('../input/dates_proj_data.csv')
        
    return total_train, total_test, total_proj, train_time, test_time, time_proj

In [32]:
total_train, total_test, total_proj, time_train, time_test, time_proj = preprocess_3d(temp_histo, temp_proj, prcp_histo,
              prcp_proj, wind_histo, wind_proj)

#### 7. Step-by-Step Preprocessing

In [13]:
temp_histo_nd, time_list = xr_to_ndarray(temp_histo, sq32_west_europe)
prcp_histo_nd, _ = xr_to_ndarray(prcp_histo, sq32_west_europe)
wind_histo_nd, _ = xr_to_ndarray(wind_histo, sq32_west_europe)

In [14]:
temp_proj_nd, time_proj = xr_to_ndarray(temp_proj, sq32_west_europe)
prcp_proj_nd, _ = xr_to_ndarray(prcp_proj, sq32_west_europe)
wind_proj_nd, _ = xr_to_ndarray(wind_proj, sq32_west_europe)

In [15]:
temp_extrema = get_extrema(temp_histo_nd, temp_proj_nd)
prcp_extrema = get_extrema(prcp_histo_nd, prcp_proj_nd)
wind_extrema = get_extrema(wind_histo_nd, wind_proj_nd)
extrema = np.array([temp_extrema, prcp_extrema, wind_extrema, temp_extrema, prcp_extrema, wind_extrema])

In [16]:
temp_histo_norm = normalize(temp_histo_nd, temp_extrema)
temp_proj_norm = normalize(temp_proj_nd, temp_extrema)

prcp_histo_norm = normalize(prcp_histo_nd, prcp_extrema)
prcp_proj_norm = normalize(prcp_proj_nd, prcp_extrema)

wind_histo_norm = normalize(wind_histo_nd, wind_extrema)
wind_proj_norm = normalize(wind_proj_nd, wind_extrema)

In [17]:
train_temp, test_temp, train_time, test_time = split_train_test(temp_histo_norm,
                                                                time_list)
train_prcp, test_prcp, _, _ = split_train_test(prcp_histo_norm, 
                                               time_list)
train_wind, test_wind, _, _ = split_train_test(wind_histo_norm,
                                               time_list)

In [18]:
total_train = ndarray_to_3d(train_temp, train_prcp, train_wind)
total_test = ndarray_to_3d(test_temp, test_prcp, test_wind)
total_proj = ndarray_to_3d(temp_proj_norm, 
                           prcp_proj_norm, 
                           wind_proj_norm)

In [ ]:
np.save("../input/preprocessed_3d_train_data.npy", total_train)
np.save("../input/preprocessed_3d_test_data.npy", total_test)
np.save("../input/preprocessed_3d_proj_data.npy", total_proj)

In [ ]:
pd.DataFrame(train_time).to_csv('../input/dates_train_data.csv')
pd.DataFrame(test_time).to_csv('../input/dates_test_data.csv')
pd.DataFrame(time_proj).to_csv('../input/dates_proj_data.csv')